## Melon Top 100
1. 100곡 노래의 제목, SongID 추출(정규표현식 사용)
2. SongID로 상세페이지를 100번 요청하여 노래 상세정보 추출
3. 100곡의 상세정보를 Json file에 저장
4. Json file을 Pandas의 DataFrame객체에 저장해서 표 데이터를 만들기
5. 표 데이터를 MariaDB에 저장

### 1. 100곡 노래의 제목, SongID 추출(정규표현식 사용)

In [36]:
import requests
from bs4 import BeautifulSoup as bs
import re

url = 'https://www.melon.com/chart/index.htm'

req_header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# 요청 보내기
res = requests.get(url, headers=req_header)
if res.ok:
    # Response 객체에 포함된 text를 추출
    html = res.text
    # 추출한 text를 파싱하기 위한 BeautifulSoup 객체생성
    soup = bs(html, 'html.parser')
    # print(soup.select('div.wrap_song_info a'))
    
    # 하위에 <a href='javascript:melon.play.playSong'>에서 playSong이 포함된 걸 찾기
    # print(soup.select("div.wrap_song_info a[href*='playSong']"))
    atag_list = soup.select("div.wrap_song_info a[href*='playSong']")

    #100곡 노래 저장 list
    song_list = []
    for idx, atag in enumerate(atag_list, 1):
        # print(idx, atag)
        
        # 한곡의 노래 저장 dict
        song_dict = dict()
        # 노래 제목
        title = atag.text
        # 노래 href추출
        href = atag["href"]
        # 정규표현식 패턴으로 ID 추출
        Id = re.search(r'(\d+)\);', href)
        # 정규표현식에서 group함수는 특정문자 제거
        song_id = Id.group(1)
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        
        # 추출한 song title과 url을 song_dict에 저장
        song_dict['title'] = title
        song_dict['url'] = song_url
        # song_dict를 song_list에 추가
        song_list.append(song_dict)
        
print(len(song_list))
song_list[:3]

100


[{'title': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)',
  'url': 'https://www.melon.com/song/detail.htm?songId=33077590'},
 {'title': '밤하늘의 별을(2020)',
  'url': 'https://www.melon.com/song/detail.htm?songId=33061995'},
 {'title': 'Dynamite',
  'url': 'https://www.melon.com/song/detail.htm?songId=32872978'}]

### 2. SongID로 상세페이지를 100번 요청하여 노래 상세정보 추출

In [45]:
from bs4 import BeautifulSoup
import re

req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

song_detail_list = []
for song in song_list:
    song_detail_dict = {}
    #print(song['title'], song['link'])
    song_url = song['url']
    res = requests.get(song_url, headers=req_header)
    print(res.status_code)
    if res.ok:
        html = res.text
        soup = BeautifulSoup(html,'html.parser')
        #노래제목
        song_detail_dict['곡명'] = song['title']
        #가수
        song_detail_dict['가수'] = soup.select('a[href*="goArtistDetail"] span')[0].text
        #앨범
        song_detail_dict['앨범'] = soup.select('div.meta dd')[0].text
        #발매일
        song_detail_dict['발매일'] = soup.select('div.meta dd')[1].text
        #장르
        song_detail_dict['장르'] = soup.select('div.meta dd')[2].text
        #가사
        lyric = soup.select('div#d_video_summary')[0].text
        regexp = re.compile(r'[\n\r\t]')        
        song_detail_dict['가사'] = regexp.sub('',lyric.strip())
        #각각의 Song정보를 담고 있는 dict를 list에 저장
        song_detail_list.append(song_detail_dict)
#         print('-----')
#         print(song_detail_dict)    

print(len(song_detail_list))
print(song_detail_list[0:3])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
100
[{'곡명': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)', '가수': '미란이 (Mirani)', '앨범': '쇼미더머니 9 Episode 1', '발매일': '2020.11.21', '장르': '랩/힙합', '가사': "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈

In [44]:
# 정규표현식 연습하기
# 문자열에 포함된 \n\r\t 제거하기

import re

my_str = 'a\nb\rc\td' * 3
print(my_str)

# 정규표현식 패턴 객체 생성
regexp = re.compile(r'[\n\r\t]')
print(type(regexp), regexp)

# 패턴과 일치하는 문자열을 찾아서 다른 문자열로 치환
result = regexp.sub(" ", my_str)
print(result)
result = result.replace(" ", "")
print(result)

a
c	da
c	da
c	d
<class 're.Pattern'> re.compile('[\\n\\r\\t]')
a b c da b c da b c d
abcdabcdabcd


### 3. 100곡의 상세정보를 Json file에 저장

In [57]:
import json

with open('data/songs.json', 'w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)

In [60]:
import json

with open('data/songs.json', 'r', encoding='utf-8') as file:
    f = file.read()
    song_json = json.loads(f)
    
song_json[:1]

[{'곡명': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)',
  '가수': '미란이 (Mirani)',
  '앨범': '쇼미더머니 9 Episode 1',
  '발매일': '2020.11.21',
  '장르': '랩/힙합',
  '가사': "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈 앞에녹슨 팔찌 버려 문 앞에구제 벨트 아직 허리에원망하던 과거와 춤출래인디고 카니발 Skrt-skrt목숨은 다 꺼내놓고자판기 밑 먼지 덕지덕지 붙은동전까지 털어-털어여기까지 왔지What you know about me내 속 이야기 털어놓으려지갑 털어 차린 술상 위넌 폰만 바라봤잖아이제 그 스크린에서 날 봐라 I'ma먹고서 Fly 너가 나갈 건 삔또지악플들이 돈과 함께 쏟아지니Got a nice ring to it꼬우면 네 오빠인지 아저씨인지한테일러바치고 한 판 더 뜨자 해 빛이나는 Solo라니까 뭘 그렇게 재니다 드루와 내 패기170에 60kg도 안 되지만 국보 1호 MCThe Q's certified new kingin the rap내가 TV에 쬐끔 나왔다고연예인 된 줄 알았다는 이미 넌 패B

### 4. Json file을 Pandas의 DataFrame객체에 저장해서 표 데이터를 만들기

In [63]:
import pandas as pd

# DataFrame 객체 생성
song_df = pd.DataFrame(columns=['곡명', '가수', '앨범', '발매일', '장르', '가사'])

for song in song_json:
    series_obj = pd.Series(song)
    song_df = song_df.append(series_obj, ignore_index=True)

In [64]:
song_df.head(5)

,곡명,가수,앨범,발매일,장르,가사
0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,2020.11.21,랩/힙합,보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아...
1,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래 너는 내가 사랑하니까 더 소중하니까 오직 너 아니면...
2,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Cos ah ahI’m in the stars tonightSo watch me b...
3,잠이 오질 않네요,장범준,잠이 오질 않네요,2020.10.24,발라드,당신은 날 설레게 만들어조용한 내 마음 자꾸만 춤추게 해얼마나 얼마나 날 떨리게 하...
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...


In [67]:
print(f'shape : {song_df.shape}')
print(f'columns : {song_df.columns}')
print(f'index : {song_df.index}')
print(f'values : {song_df.values[:1]}')

shape : (100, 6)
columns : Index(['곡명', '가수', '앨범', '발매일', '장르', '가사'], dtype='object')
index : RangeIndex(start=0, stop=100, step=1)
values : [['VVS (Feat. JUSTHIS) (Prod. GroovyRoom)' '미란이 (Mirani)'
  '쇼미더머니 9 Episode 1' '2020.11.21' '랩/힙합'
  "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈 앞에녹슨 팔찌 버려 문 앞에구제 벨트 아직 허리에원망하던 과거와 춤출래인디고 카니발 Skrt-skrt목숨은 다 꺼내놓고자판기 밑 먼지 덕지덕지 붙은동전까지 털어-털어여기까지 왔지What you know about me내 속 이야기 털어놓으려지갑 털어 차린 술상 위넌 폰만 바라봤잖아이제 그 스크린에서 날 봐라 I'ma먹고서 Fly 너가 나갈 건 삔또지악플들이 돈과 함께 쏟아지니Got a nice ring to it꼬우면 네 오빠인지 아저씨인지한테일러바치고 한 판 더 뜨자 해 빛이나는 Solo라니까 뭘 그렇게 재니다 드루와 내 패기

In [69]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   곡명      100 non-null    object
 1   가수      100 non-null    object
 2   앨범      100 non-null    object
 3   발매일     100 non-null    object
 4   장르      100 non-null    object
 5   가사      100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [70]:
song_df['가수'].value_counts().head()

방탄소년단               7
아이유                 5
릴보이 (lIlBOI)        4
폴킴                  4
머쉬베놈 (MUSHVENOM)    3
Name: 가수, dtype: int64

In [71]:
song_df['가수'].unique()

array(['미란이 (Mirani)', '경서', '방탄소년단', '장범준', '릴보이 (lIlBOI)', '임창정',
       'BLACKPINK', '산들', '스탠딩 에그', 'Jawsh 685', '규현 (KYUHYUN)', '적재',
       'AKMU (악동뮤지션)', '아이유', '박진영', '벤', '노을', '오반 (OVAN)',
       'Khundi Panda', '마마무 (Mamamoo)', '폴킴', '조정석', '창모 (CHANGMO)',
       '오마이걸 (OH MY GIRL)', '순순희', '환불원정대', '스윙스', '경서예지', 'TWICE (트와이스)',
       '마크툽 (MAKTUB)', '백지영', '화사 (Hwa Sa)', '이승기', 'Maroon 5',
       '가호 (Gaho)', '전상근', '이예준', '머쉬베놈 (MUSHVENOM)', '이하이', '신예영',
       '제시 (Jessi)', 'Sia', '잔나비', 'Mariah Carey', '태연 (TAEYEON)',
       'Tones And I', '황인욱', '블루 (BLOO)', 'Ariana Grande', 'Anne-Marie',
       '먼데이 키즈 (Monday Kiz)', '비', 'IZ*ONE (아이즈원)', '엠씨더맥스 (M.C the MAX)',
       '송하예', '전미도', '임영웅', '#안녕', '정준일', 'Dua Lipa', '허각', '지코 (ZICO)',
       '싹쓰리 (유두래곤, 린다G, 비룡)', '청하', '원슈타인', 'Etham', 'Crush',
       'Conan Gray', 'Lauv', 'aespa', '윤도 (YoonDo)', '양다일', '성시경'],
      dtype=object)

### 특정 행과 열을 선택
* loc[], iloc[] 사용
    - Slicing 사용해서 구간을 선택
    - 특정행과 특정열을 선택
    - 조건식을 만족하는 행과 열을 선택

In [73]:
# loc[] - location의 약자, 특정행과 열을 선택할때 사용
# loc[행을 선택하는 기준, 열을 선택하는 기준] 

# 인덱스가 0 부터 5 까지인 행과 모든 열
song_df.loc[0:5, :]

,곡명,가수,앨범,발매일,장르,가사
0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,2020.11.21,랩/힙합,보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아...
1,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래 너는 내가 사랑하니까 더 소중하니까 오직 너 아니면...
2,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Cos ah ahI’m in the stars tonightSo watch me b...
3,잠이 오질 않네요,장범준,잠이 오질 않네요,2020.10.24,발라드,당신은 날 설레게 만들어조용한 내 마음 자꾸만 춤추게 해얼마나 얼마나 날 떨리게 하...
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...
5,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다,2020.10.19,발라드,내가 널 떠났어야 했는데 왜 떠나야 하는지도 아는데 어떤 아무 말도 아무것도 줄 게...


In [74]:
# Slicing으로 인덱스가 0부터 8까지, 1줄 씩 skip
song_df.loc[0:8:2, :]

,곡명,가수,앨범,발매일,장르,가사
0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,2020.11.21,랩/힙합,보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아...
2,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Cos ah ahI’m in the stars tonightSo watch me b...
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...
6,Lovesick Girls,BLACKPINK,THE ALBUM,2020.10.02,댄스,영원한 밤창문 없는 방에 우릴 가둔 loveWhat can we say매번 아파도 ...
8,Life Goes On,방탄소년단,BE,2020.11.20,랩/힙합,어느 날 세상이 멈췄어아무런 예고도 하나 없이봄은 기다림을 몰라서눈치 없이 와버렸어...


In [86]:
# 아이유의 노래, 앨범, 장르
song_df[song_df["가수"] == '아이유'][['곡명', '앨범', '장르']]

,곡명,앨범,장르
16,에잇(Prod.&Feat. SUGA of BTS),에잇,록/메탈
30,Blueming,Love poem,록/메탈
36,마음을 드려요,사랑의 불시착 OST Part 11,"발라드, 국내드라마"
73,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),Real,댄스
75,Love poem,Love poem,록/메탈


In [92]:
# 장르가 댄스인 곡명부터 장르까지 가져오기
song_df.loc[song_df['장르'] == '댄스', '곡명':'장르'].sort_values(by='발매일', ascending=False).reset_index(drop=True)

,곡명,가수,앨범,발매일,장르
0,나로 바꾸자 (duet with JYP),비,나로 바꾸자 (duet with JYP),2020.12.31,댄스
1,Panorama,IZ*ONE (아이즈원),One-reeler / Act IV,2020.12.07,댄스
2,Black Mamba,aespa,Black Mamba,2020.11.17,댄스
3,I CAN’T STOP ME,TWICE (트와이스),Eyes wide open,2020.10.26,댄스
4,딩가딩가 (Dingga),마마무 (Mamamoo),딩가딩가 (Dingga),2020.10.20,댄스
5,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME,2020.10.10,댄스
6,Lovesick Girls,BLACKPINK,THE ALBUM,2020.10.02,댄스
7,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스
8,When We Disco (Duet with 선미),박진영,When We Disco,2020.08.12,댄스
9,다시 여기 바닷가,"싹쓰리 (유두래곤, 린다G, 비룡)",다시 여기 바닷가,2020.07.18,댄스


### 5. 표 데이터를 MariaDB에 저장